# Programmer Pact

The [Programmer Pact](./ProgrammerPact_Python_2026.pdf) is a set of basic expectations for code structure and style. In this notebook, I explain the Pact's expectations with examples.


## `break` and `continue` shall be used only on days the Cubs defeat the Cardinals 7-2.

The odds that the Cubs beat the Cardinals **7–2** are about 1%. That’s also about how often you should need `break` or `continue` in this course: essentially never.

Why avoid them? Used casually, `break`/`continue` create extra control-flow paths that make code harder to reason about and harder to test. In spirit, they behave a bit like a `goto`: they jump execution to a different place than the loop structure suggests.

Instead, write loops whose stopping logic is expressed **in the loop condition** (or via a clean boolean flag) so the “shape” of the code matches its logic.

In [ ]:
friends = ["Eleven", "Max", "Mike", "Dustin", "Will", "Lucas", "Nancy"]


def find_friend(friend_name):
    found = False
    for i in range(len(friends)):
        if friends[i] == friend_name:
            found = True
            break
    return found

What happens above is that when `friends[i] == friend_name` evaluates to `True`, the `break` statement instructs the program to go to the fist line outside the for-loop - in this case, to the line with the `return` statement. In other words, `break` is essentially a [`go to`](https://en.wikipedia.org/wiki/Goto) statement. Such statements create program paths that may be easy to write in the moment but not clear to follow their logic. A better version of the method above is:


In [ ]:
friends = ["Eleven", "Max", "Mike", "Dustin", "Will", "Lucas", "Nancy"]


def find_friend(friend_name):
    found = False
    i = 0
    while i < len(friends) and not found:
        found = friends[i] == friend_name
        i += 1
    return found

In the code above, we use a `while` loop that ends when one of two things happen: we run out of list elements to consider or we find the list element we are looking for. The logic of ending the loop is built in the loop statement. There is no need to artificially `break` the loop when we find the friend we are looking for. The loop will end as a result of finding the value we are searching for.


## Methods that return values shall have only one `return` statement

This requirement implements the _single-entry single-exit_ principle in structure programming. Consider the following example that returns the position of a value within a list, or -1 if the value is not found.


In [ ]:
friends = ["Eleven", "Max", "Mike", "Dustin", "Will", "Lucas", "Nancy"]


def index_of_friend(friend_name):
    for i in range(len(friends)):
        if friends[i] == friend_name:
            return i
    return -1

The code works fine but 50% of it is just `return` statements. An alternative version, with a single return statement is:


In [ ]:
friends = ["Eleven", "Max", "Mike", "Dustin", "Will", "Lucas", "Nancy"]


def index_of_friend(friend_name):
    index = -1
    for i in range(len(friends)):
        if friends[i] == friend_name:
            index = i
    return index

Strictly speaking, we do not need a `for` loop for the method above. As soon as a match is found, the loop must end. This can be accomplished with a `break` statement right after the assignment `index=i`. But the use of `break` is _highly_ discouraged. Instead we can write a `while` loop as follows.


In [ ]:
friends = ["Eleven", "Max", "Mike", "Dustin", "Will", "Lucas", "Nancy"]


def index_of_friend(friend_name):
    index = -1
    i = 0
    while i < len(friends) and index == -1:
        if friends[i] == friend_name:
            index = i
        i += 1
    return index

The code above is a bit longer than the first version with the two return statements. However, the additional lines here can be justified by the fact that method `find_friend` that we wrote earlier, can be simplified to:


In [ ]:
friends = ["Eleven", "Max", "Mike", "Dustin", "Will", "Lucas", "Nancy"]


def find_friend(friend_name):
    return index_of_friend(friend_name) != -1

As your programming skills develop, you will be able to use multiple returns in a method. In fact, two or more return statements are essential for code readability when a method has more than 20 or 30 lines of code -- comment lines don't count! Between readability and a rigorous adherence to the _single-entry single-exit_ principle, we side with readability. For now, most of our methods are less than 20 lines of actual code, and a single return statement is plenty.


## Methods that return no value shall have no `return` statements.

Some methods do not return values. For example:


In [ ]:
friends = ["Eleven", "Max", "Mike", "Dustin", "Will", "Lucas", "Nancy"]


def describe(list_of_friends):
    print(f"There are {len(list_of_friends)} friends in the list.")
    if len(list_of_friends) > 0:
        print(f"The first friend in the list is {list_of_friends[0]}.")
        print(
            f"The last friend in the list is {list_of_friends[len(list_of_friends) - 1]}."
        )
    else:
        print("The list is empty.")
    return  # not really needed here.

There is no reason to end the method with a `return` statement. The method will end, organically, when its last line is executed.


## All class variables shall be \_private


In object-oriented design, variables within a class are _private,_ i.e., not directly accessible by other classes. For example, consider the following simple class.


In [ ]:
class Friend:
    def __init__(self, name):
        self.__name = name

Adding two underscores before a class variable, in this case `self.__name` instead of `self.name` makes the variable a bit difficult to access directly. For example,


In [ ]:
my_best_friend = Friend("Vekna")
print(my_best_friend.__name)  # This will raise an AttributeError

The double underscore _mangles_ the variable name into something more complex. In this case the variable `__name` is mangled into `_Friend__name`. The standard technique to access a private variable's value is to write a method:


In [ ]:
class Friend:
    def __init__(self, name):
        self.__name = name

    def get_name(self):
        return self.__name


my_best_friend = Friend("Vekna")
print(my_best_friend.get_name())  # This will print "Vekna"

Methods that allow access to private variables are called _accessors_ or _getter methods._

In Python, there is a workaround private variables but it is not recommended.


In [ ]:
print(my_best_friend._Friend__name)  # This will print "Vekna" but is not recommended

Basically, when we design our own objects we want to control under what conditions data can leave and enter the object. Accessor methods control how data leave the object. Conversely, _mutator_ or _setter methods_ control how data enter the object. For example,


In [ ]:
class Friend:
    def __init__(self, name):
        self.__name = name

    def get_name(self):  # getter method
        return self.__name

    def set_name(self, name):  # setter method
        self.__name = name

It is also acceptable to use a single underscore to indicate a private member of a class.


In [ ]:
class Friend:
    def __init__(self, name):
        self._name = name

    def get_name(self):  # getter method
        return self._name

    def set_name(self, name):  # setter method
        self._name = name

A single underscore will not mangle the variable name. However, it's a guideline and a request to other programs to treat the class member as private. The guiding principle here, attributed to Guido van Rossum, the creator of Python, is that we are all adults and we should respect the design choices other programmers have made when we use their code.


In [ ]:
my_best_friend = Friend("Vekna")
print(my_best_friend.get_name())  # This will print "Vekna"
print(my_best_friend._name)  # This will print "Vekna" but is not recommended

## Code shall always have comments

There are two forms of comments required. One is the _docstring._ The other is _single_ or _multi-line comments._ For more information read the [comments section](https://peps.python.org/pep-0008/#comments) section of Python Enhancement Proposal 8 (PEP8).

### Docstrings

Docstrings are blocks of text surrounded by triple _double_ quotes: `"""..."""`. They appear immediately after the header of a method and serve as the method's manual. Docstrings describe what a method does, how it does, what it requires as input, and what it produces as output. They can be quite extensive. Docstrings may appear only after the method header. They can also appear after the declaration of a class. These are the only two places where docstrings are permitted.

### Line comments

These are lines that begin with the hash symbol `#` and provide local narrative for the logic or the rationale of code.

### Example of comments

Here's the earlier method `index_of_friend` appropriately commented.


In [ ]:
friends = ["Eleven", "Max", "Mike", "Dustin", "Will", "Lucas", "Nancy"]


def index_of_friend(friend_name):
    """
    Returns the index of the friend_name in the friends list.
    In case of multiple occurrences, returns the index of the first one.
    If the friend is not found, returns -1.

    Input:
    ------
    friend_name: str
        Name of the friend to search for.

    Returns:
    --------
    int:
        index of friend_name in friends or -1 if not found.
    """
    # Assume friend is not found.
    index = -1
    # Iterate through the friends list.
    i = 0
    # Loop until the end of the list or until the friend is found.
    while i < len(friends) and index == -1:
        # Check if the current friend matches friend_name
        if friends[i] == friend_name:
            # Match found, set index to current position;
            # This will cause the loop to end.
            index = i
        # Proceed to the next friend.
        i += 1
    # Done. Index at this point is either -1 (not found) or
    # the index of friend_name.
    return index

## Code will be neat and clean

This requirement comes down to four basic rules.

1. **Name formatting**  
   For variables and functions we use `snake_case_style`. For classes we use `PascalCase`. For constants we use `SCREAMING_SNAKE_CASE`. And for internal use we prefix the name with one or two underscores. For more details read the [naming section](https://peps.python.org/pep-0008/#naming-conventions) in Python Enhancement Proposal 8 (PEP8).

1. **Blank lines**  
   Blank lines inside a method should be used sparingly. My suggestion is that for methods with 20 or fewer lines of actual code, blank lines should be avoided. For more details read the [blank lines section](https://peps.python.org/pep-0008/#blank-lines) in Python Enhancement Proposal 8 (PEP8).

1. **79 character lines**  
   Each line should not exceed 79 characters in length. Most editors allow users to set the line length limit and show a vertical line at the corresponding column. If such feature is not available, you may use a comment line that is 79 characters long as a ruler:

   ```python
   #234567890123456789012345678901234567890123456789012345678901234567890123456789
   ```

   For more details read the [maximum line length section](https://peps.python.org/pep-0008/#maximum-line-length) in Python Enhancement Proposal 8 (PEP8).

1. **Use type hints**  
   When possible, use [simple type hints](https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html). Simple here means hints that do not require an imported module or package.


## No unauthorized imports

Many assignment problems can be solved with advanced methods that are available in various Python modules and packages. At this point in your training, the objective is to develop your own problem solving skills instead of applying those of others. Therefore, the use of the `import` statement is restricted to the following modules when necessary:

```python
import random
import time
```

Occasionally, other modules and packages may be allowed and they'll be mentioned explicitly in the assignment specifications.

If you feel necessary to import a module or a package not explictly allowed, you must first obtain permission from your instructor. The rejection rate of such requests exceeds 150%.


## Code shall execute

Any code that you turn in, should execute without runtime errors. Ideally it should also produce the correct results. Code that does not execute will receive no credit. For example, you are asked to write a function that reports the parity of a number. The three possible scenarios are shown below, with type hints.


In [ ]:
# DOESN'T RUN because of syntax error
def parity_check_with_syntax_error(number:int) -> str:
    result = "Odd"
    if number % 2 = 0:
        result = "Even"
    return result

In [ ]:
# RUNS but produces incorrect result
def parity_check_flawed_logic(number: int) -> str:
    result = "Odd"
    if number % 2 == 1:
        result = "Lemongrass"
    return result

In [ ]:
# RUNS and produces correct result
def parity_check_correct(number: int) -> str:
    result = "Odd"
    if number % 2 == 0:
        result = "Even"
    return result

## No magic values

A magic value is a literal that is _not_:

- `-1`, `0`, `1`
- `-1.0`, `0.0`, `1.0`
- `2` in determing parity
- `""` (the empty string)
- `" "` (a single space string)
- a boolean value (`True`, `False`)
- the `None` object.
- a numeric value in a mathematical formula.

Any value that does not fall in one of the categories above should be assigned either to a variable or a constant.

For example, the expession

```python
s = s_0 + v_0 * t + 0.5 * a * t ^ 2
```

does not contain magic values, despite the literals `0.5` and `2`, because it implements the formula for Newton's second law of motion: $s=s_0+v_0t+\frac{1}{2}at^2$.

However, the parity method from the previous example,

```python
def parity_check_correct(number: int) -> str:
    result = "Odd"
    if number % 2 == 0:
        result = "Even"
    return result
```

contains the magic values `"Odd"` and `"Even"`. These values are best to be assigned in constants that are then accessible to the method, as shown below.


In [ ]:
ODD = "Odd"
EVEN = "Even"


def parity_check_correct(number: int) -> str:
    result = ODD
    if number % 2 == 0:
        result = EVEN
    return result

It may seem weird, to replace a string literal with a constant whose name matches the literal. It just so happens in this example that the constant names and their content are very similar. Consider another example


In [ ]:
def is_upper_case_letter(char: str) -> bool:
    return len(char) == 1 and ord(char) >= 65 and ord(char) <= 90

What are the literals `65` and `90`? To the trained eye, they are the ASCII values for upper case letters `A` and `Z`. Why not show this more clearly?


In [ ]:
LETTERS = 26  # Number of letters in the English alphabet
ASCII_A = 65  # ASCII value of 'A' from the ASCII table (Wikipedia etc)
ASCII_Z = ASCII_A + LETTERS - 1  # Why -1? A comment here would be helpful.


def is_upper_case_letter(char: str) -> bool:
    """Full docstring ommitted for brevity."""
    # Comment explaining the logic for len(char)==1 would be helpful.
    return len(char) == 1 and ord(char) >= ASCII_A and ord(char) <= ASCII_Z

# Why is the Pact important?

The Programmer’s Pact is a set of rules designed to promote readability, consistency, and long-term maintainability of code. The Pact is a deliberately abbreviated and pragmatic subset of PEP 8, the primary style guide for writing Python code. Style guides matter because they establish shared conventions that allow code to be understood, maintained, and extended by others—often long after the original author has moved on.

A widely accepted principle in software engineering is that code is read far more often than it is written—an idea closely associated with Donald Knuth, Martin Fowler, and Guido van Rossum. Readability is therefore not cosmetic; it is foundational. Style guides support readability by setting clear expectations about naming, structure, and layout, so readers know where to look, what to expect, and how to interpret what they see.

A secondary reason for enforcing the Pact in my courses is practical. Consistent style makes it easier to distinguish carefully written student work from rushed, copied, or automatically generated code. Submissions that substantially violate the Pact often indicate a lack of engagement with the assignment and, in some cases, raise concerns about academic integrity.